<a href="https://colab.research.google.com/github/alexswcr/Face-Landmark-Detection-with-CNNs/blob/Add-Files/Task_2_Face_Alignment_Pt_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Alignment NoteBook 2: Creating Models

**To run these codeblocks, please replace the file path in np.load() to your directory containing the respective file, the code submission zip folder should contain a copy of each file needed**

This notebook shows how each model was created, this was made into a separate notebook due to RAM running out on the first notebook, meaning all codeblocks could not be run in excession.

Instead, the preprocessed data was saved into a .npz file, so that it could be loaded in this notebook and used to train the model.

Each model is set to run for 15 epochs, which takes around 7-8hrs, I would not advise trying to run multiple of the 'Creating Model x' code blocks in the same runtime as the RAM will likely run out. This is why each code block has repeated code, they are designed to be run independently (after this first code block to get the training data).

## Splitting training data 90/10%

In [ ]:
from google.colab import drive
import numpy as np
import keras
from keras import layers
from tqdm import tqdm
import gc
import tensorflow as tf

drive.mount("/content/gdrive/")

#Calculated mean landmark distance for a batch
@keras.saving.register_keras_serializable(package="my_package", name="landmark_distance")
def landmark_distance(y_true,y_pred):

  distance = tf.sqrt(tf.reduce_sum(tf.square(y_true-y_pred),-1))

  return tf.reduce_mean(distance)


#Load training Data
transform_data = np.load('/content/gdrive/MyDrive/Colab-Notebooks/AML-coursework/PROCESSED_DATA_FINAL.npz',allow_pickle=True)
x = transform_data['x_train']
y = transform_data['y_train']

#Split training data
train_test_split = int(np.round(x.shape[0]*0.9))
x_train = x[:train_test_split]
y_train = y[:train_test_split]
x_test = x[train_test_split:]
y_test = y[train_test_split:]

## Creating Model 1
Please do not run the commented code as this will overwrite the trained model and replace it with a new one.

In [ ]:
#Size of an image with batch size 1
input_shape = (180,180,1)

#Defining the network
model = keras.Sequential(
    [
        #Set input shape
        keras.Input(shape = input_shape),
        #Convolutional layers and pooling layers, convolutional layer uses relu activation
        layers.Conv2D(64, kernel_size=(3, 3),kernel_initializer='he_normal', activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3),kernel_initializer='he_normal', activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, kernel_size=(3, 3),kernel_initializer='he_normal', activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, kernel_size=(3, 3),kernel_initializer='he_normal', activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),

        #Flatten outputs
        layers.Flatten(),
        #Dropout to reduce overfitting
        layers.Dropout(0.2),
        #Produce 10 outputs
        layers.Dense(10),
        #Reshape outputs
        layers.Reshape((5,2))
    ]
)
#Show model output sizes and parameters at each layer
model.summary()

batch_size = 32
epochs = 15

#Compile model with loss and optimizer function, showing accuracy and mean landmark_distance
model.compile(loss = "mean_squared_error",optimizer="adam",metrics= ["accuracy",landmark_distance])

#Save model with the highest validation accuracy 
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath="/content/gdrive/MyDrive/Colab-Notebooks/AML-coursework/Best_CNN_model_New_Copy.keras",monitor="val_accuracy",mode="max",save_best_only=True)

#GUI to show training performance and progress
tensorBoard = keras.callbacks.TensorBoard(
    log_dir="/content/gdrive/MyDrive/Colab-Notebooks/AML-coursework/CNN_Model_Logs",
    histogram_freq=0,
    embeddings_freq=0,
    update_freq="epoch",
)
#Save some RAM with garbage collection
gc.collect()

#Load tensorBoard
%tensorboard --logdir logs

#Train model on training data
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1,callbacks=(model_checkpoint,tensorBoard))

{-45: 0.7071067811865476, -44: 0.7193398003386512, -43: 0.7313537016191705, -42: 0.7431448254773942, -41: 0.754709580222772, -40: 0.766044443118978, -39: 0.7771459614569709, -38: 0.7880107536067219, -37: 0.7986355100472928, -36: 0.8090169943749475, -35: 0.8191520442889918, -34: 0.8290375725550416, -33: 0.838670567945424, -32: 0.848048096156426, -31: 0.8571673007021123, -30: 0.8660254037844387, -29: 0.8746197071393957, -28: 0.882947592858927, -27: 0.8910065241883679, -26: 0.898794046299167, -25: 0.9063077870366499, -24: 0.9135454576426009, -23: 0.9205048534524404, -22: 0.9271838545667874, -21: 0.9335804264972017, -20: 0.9396926207859084, -19: 0.9455185755993168, -18: 0.9510565162951535, -17: 0.9563047559630354, -16: 0.9612616959383189, -15: 0.9659258262890683, -14: 0.9702957262759965, -13: 0.9743700647852352, -12: 0.9781476007338057, -11: 0.981627183447664, -10: 0.984807753012208, -9: 0.9876883405951378, -8: 0.9902680687415704, -7: 0.992546151641322, -6: 0.9945218953682733, -5: 0.996194

## Creating Model 2

In [ ]:
#Pretty much identical to Model 1 except for a different number of layers
input_shape = (180,180,1)

batch_size = 32
epochs = 50


model = keras.Sequential(
    [
        keras.Input(shape = input_shape),
        layers.Conv2D(64, kernel_size=(3, 3),kernel_initializer='he_normal', activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, kernel_size=(3, 3),kernel_initializer='he_normal', activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(10),
        layers.Reshape((5,2))
    ]
)
model.summary()

batch_size = 32
epochs = 15

model.compile(loss = "mean_squared_error",optimizer="adam",metrics= ["accuracy",landmark_distance])
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath="/content/gdrive/MyDrive/Colab-Notebooks/AML-coursework/CNN_Model_2_Reduced_layers.keras",monitor="val_accuracy",mode="max",save_best_only=True)

tensorBoard = keras.callbacks.TensorBoard(
    log_dir="/content/gdrive/MyDrive/Colab-Notebooks/AML-coursework/CNN_Model_Logs",
    histogram_freq=0,
    embeddings_freq=0,
    update_freq="epoch",
)
gc.collect()
%load_ext tensorboard
%tensorboard --logdir logs
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1,callbacks=(model_checkpoint,tensorBoard))

##Creating Model 3

In [ ]:
#Pretty much identical to Model 1 except for a different number of filters
input_shape = (180,180,1)

batch_size = 32
epochs = 50


model = keras.Sequential(
    [
        keras.Input(shape = input_shape),
        layers.Conv2D(32, kernel_size=(3, 3),kernel_initializer='he_normal', activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, kernel_size=(3, 3),kernel_initializer='he_normal', activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3),kernel_initializer='he_normal', activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3),kernel_initializer='he_normal', activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(10),
        layers.Reshape((5,2))
    ]
)
model.summary()

batch_size = 32
epochs = 15

model.compile(loss = "mean_squared_error",optimizer="adam",metrics= ["accuracy",landmark_distance])
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath="/content/gdrive/MyDrive/Colab-Notebooks/AML-coursework/CNN_Model_3_Reduced_filters.keras",monitor="val_accuracy",mode="max",save_best_only=True)

tensorBoard = keras.callbacks.TensorBoard(
    log_dir="/content/gdrive/MyDrive/Colab-Notebooks/AML-coursework/CNN_Model_Logs",
    histogram_freq=0,
    embeddings_freq=0,
    update_freq="epoch",
)
gc.collect()
%load_ext tensorboard
%tensorboard --logdir logs
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1,callbacks=(model_checkpoint,tensorBoard))